In [1]:
from pyspark.sql import SparkSession
import os
import configparser

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1676819506019_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_song = spark.read.json("s3a://udacity-dend/song_data/A/A/*/*.json") # I am using less data to be faster to run

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_song.printSchema()
df_song.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

+------------------+---------------+--------------------+----------------+--------------------+----------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|         artist_name|  duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+--------------------+----------+---------+------------------+--------------------+----+
|ARSUVLW12454A4C8B8|       35.83073|           Tennessee|       -85.97874|Royal Philharmoni..

In [ ]:
#  df[df['page']== 'NextSong']

In [19]:
# df_song.createOrReplaceTempView("songs")
# df_songs_table= spark.sql("""
#     SELECT DISTINCT(song_id) AS song_id,
#        title,                                          
#        artist_id,
#        year,
#        duration                           
# FROM songs
# """).toPandas()

df_song_parquet = df_song.select(['song_id', 'title', 'artist_id', 'year', 'duration'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df_song_parquet.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(song_id='SOBTCUI12A8AE48B70', title='Faust: Ballet Music (1959 Digital Remaster): VI.     Variations du miroir (Allegretto)', artist_id='ARSUVLW12454A4C8B8', year=0, duration=94.56281), Row(song_id='SOXRPUH12AB017F769', title='Exodus: Part I: Moses and Pharaoh', artist_id='ARXQC081187FB4AD42', year=0, duration=1047.71873), Row(song_id='SOVNKJI12A8C13CB0D', title="Take It To Da House (Featuring The Slip N' Slide Express) (Super Clean Album Version)", artist_id='ARWUNH81187FB4A3E0', year=2001, duration=227.10812), Row(song_id='SOAFBCP12A8C13CC7D', title='King Of Scurf (2007 Digital Remaster)', artist_id='ARTC1LV1187B9A4858', year=1972, duration=301.40036), Row(song_id='SOKTJDS12AF72A25E5', title='Drown In My Own Tears (24-Bit Digitally Remastered 04)', artist_id='ARA23XO1187B9AF18F', year=0, duration=192.522)]

In [21]:
#df_song_parquet.write.parquet("s3a://#####/songs/df_songs_table.parquet") ##not partitioned

df_song_parquet.write.parquet(s3a://#####/songs/df_songs_table.parquet",partitionBy=['year', 'artist_id'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df_artists = df_song.select(['artist_id','artist_name','artist_location','artist_latitude','artist_longitude'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df_artists.write.parquet("s3a://#####/df_artists_table.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
df_log = spark.read.json("s3a://udacity-dend/log_data/2018/11/*.json") # I am using less data to be faster to run

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
df_log.printSchema()
df_log.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

+--------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+-------------+------+-------------+--------------------+------+
|  artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|m

In [34]:
## pandas command  df = df[df['page']== 'NextSong']
 # filter by NextSong action
df_log2 = df_log.where(df_log.page == 'NextSong')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
import  pyspark.sql.functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:

df_log2 = df_log2.withColumn("ts", F.to_timestamp(F.col("ts")/1000))
df_log2.printSchema()
df_log2.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: timestamp (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+--------------------+--------------------+------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|lev

In [38]:
users_table = df_log2.select(['userId','firstName','lastName','gender','level'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
users_table.write.parquet("s3a://#####/users/df_users_table.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
df_log2.createOrReplaceTempView("logs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
df_log2 = df_log2.withColumn("ts_dayofweek", F.dayofweek("ts"))
df_log2 = df_log2.withColumn("ts_hour", F.hour("ts"))
df_log2 = df_log2.withColumn("ts_day", F.dayofmonth("ts"))
df_log2 = df_log2.withColumn("ts_week", F.weekofyear("ts"))
df_log2 = df_log2.withColumn("ts_month", F.month("ts"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
time_table = df_log2.select(['ts','ts_hour','ts_day','ts_week','ts_month','ts_dayofweek'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
time_table.write.parquet('s3a://#####/time/time_table.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
df_total = df_log2.join(df_song, (df_log2["song"] == df_song["title"]) &
   ( df_log2["length"] == df_song["duration"]),"left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
df_total.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: timestamp (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- ts_dayofweek: integer (nullable = true)
 |-- ts_hour: integer (nullable = true)
 |-- ts_day: integer (nullable = true)
 |-- ts_week: integer (nullable = true)
 |-- ts_month: integer (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- a

In [50]:
song_play = df_total.select(['ts','userId','level','song_id','artist_id','sessionId','location','userAgent'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
song_play.write.parquet('s3a://#####/songplays/songplays_table.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…